Student ID, name.

111062201 林威盛

Preprocess Data

Personally, I extract time stamp, length of content, number of images, num of external url link, channel. Meanwhile apply bag of words in topic, author,and title, and use them as feature.

For time stamp, use re.find<tag> to get the string and use string split to get the year, month, day, hour, minute, second. Then use datetime to convert the string to weekday.
As for channel, use re.find<tag> to get the string and call funmction pd.Categorical to convert the string to number as feature.

For the length of content, number of images, num of external url link, use Bs4 to locate the tag and count them.

As for the bag of words, use Bs4 to get the string in <tag>. Then finally use CountVectorizer applying tokenizer_stem_nostop to convert the words to feature.

One crucial point is that the number of author is not always just one, so use split and append all the names in <author_name>, <article-info> into a list.
I firstly handle author name by CategoryEncoder, but the result is not good. 
And I find that the author name is not always the same, so I think use BOW will be better.



In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import numpy as np
import scipy as sp
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
import lightgbm as lgb
from lightgbm import LGBMClassifier, LGBMRegressor
import ast
from sklearn.metrics import roc_auc_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_validate
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression

df = pd.read_csv('./train.csv')
# print(df.head(2))
# # print(df.loc[0,'Page content'])

def preprocessor(text):
    # remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()

    # regex for matching emoticons, keep emoticons, ex: :), :-P, :-D
    r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
    emoticons = re.findall(r, text)
    text = re.sub(r, '', text)

    # convert to lowercase and append all emoticons behind (with space in between)
    # replace('-','') removes nose of emoticons
    text = re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-','')
    return text

def tag_extractor(text):
    soup = BeautifulSoup(text, 'html.parser')
    
    # find author
    article_info = soup.head.find('div', {'class': 'article-info'})
    author_name = article_info.find('span', {'class': 'author_name'})
    if author_name != None:
        author = author_name.get_text()
    elif article_info.span != None:
        author = article_info.span.string
    else:
        author = article_info.a.string
        
    # clean author
    author = re.sub('\s+', ' ', author.strip().lower())
    if author.startswith('by '):
        author = author[3:]
    author = re.sub('&.*;', '&', author.replace(' and ', ' & '))

    author_list = []
    if author.find(',') == -1:
        author_list = re.split('\s*&\s*', author)
    else:
        authors = re.split('\s*,\s*', author)
        if authors[-1].find('&') == -1 or len(authors[-1].split('&')[-1].strip().split()) > 3:
            author_list.append(authors[0])
        else:
            author_list += authors[:-1]
            author_list += re.split('\s*&\s*', authors[-1])
    author = ' '.join([re.sub('\s+', '_', a) for a in author_list])

    # find content
    content = soup.body.find('section', {'class': 'article-content'}).get_text()
    content_len = len(content)

    # find image
    num_image = len(soup.body.find_all('img'))

    # find a
    num_a = len(soup.body.find_all('a'))
    
    # find title
    title = soup.body.h1.string.strip().lower()
    
    # find topic
    a_list = soup.body.find('footer', {'class': 'article-topics'}).find_all('a')
    topic_list = [a.string.strip().lower() for a in a_list]
    topic = ' '.join([re.sub('\s+', '_', t) for t in topic_list])

    return author, content_len, num_image, num_a, title, topic


nltk.download('stopwords')
stop = stopwords.words('english')


def tokenizer_stem_nostop(text):
    porter = PorterStemmer()
    return [porter.stem(w) for w in re.split('\s+', text.strip()) \
            if w not in stop and re.match('[a-zA-Z]+', w)]


'''
`count.inverse_transform(np.ones((1, bag_cnts.shape[0])))[0]` returns a list of all vocabulary in the corpus
`bag_cnts.argsort()[::-1]` returns a list of indices of sorted bag_cnts in descending order (This List of Indices could sort `bag_cnts` in descending order)
'''
def get_sentiment(text):
    

    analyzer = SentimentIntensityAnalyzer()
    
    scores = analyzer.polarity_scores(text)

    return scores['pos']
    
nltk.download('vader_lexicon')

def feature_extractor(df):
    '''
    @param df: DataFrame
    '''
    # df : Id, Popularity, Page content (Popularity is the target variable to predict)
    # I want to get the number of url, the number of images and the publish year,month,day, hour, and weekday,and which channel the article is in
    # Implement your code here
    
    df_tag = []
    for i in range(len(df)):
        df_tag.append(tag_extractor(df.loc[i,'Page content']))
    
    df_tag = pd.DataFrame(df_tag, columns=['author','content_len', 'num_image', 'num_url', 'title', 'topic'])
    # df_tag['author'] = pd.Categorical(df_tag['author']).codes
    
    # df_tag.to_csv('author.csv', columns=['author'], index=False)

    print("df_tag done")
    
    # The input has <time> tag <time datetime="2014-09-19T02:07:46+00:00">2014-09-19 02:07:46 UTC</time>
    tmp_time = df['Page content'].apply(lambda x: re.findall(r'<time datetime=".*?">(.*?)</time>', x))
    
    # Some items in tmp_time are something like this:"['2014-12-11 12:52:05 UTC', 'Dec 12, 2014 at 1:36am PST', 'Dec 12, 2014 at 1:37am PST', 'Dec 12, 2014 at 1:56am PST', 'Dec 12, 2014 at 2:05am PST', 'Dec 12, 2014 at 2:11am PST']"
    tmp_time = tmp_time.apply(lambda x: x if type(x) == list else ast.literal_eval(x)[0])
    
    # tmp_time is like :"2013-06-19 15:04:30 UTC" , handle the case when tmp_time is empty
    for i in range(len(tmp_time)):
        if type(tmp_time[i]) == list and len(tmp_time[i]) ==  0:
            tmp_time[i] = ['2013-06-19 15:04:30 UTC']
            
    
    df['year'] = tmp_time.apply(lambda x: int(x[0].split('-')[0]))
    df['month'] = tmp_time.apply(lambda x: int(x[0].split('-')[1]))
    df['day'] = tmp_time.apply(lambda x: int((x[0].split('-')[2]).split(' ')[0]))
    df['hour'] = tmp_time.apply(lambda x: int((x[0].split(' ')[1]).split(':')[0]))
    df['minute'] = tmp_time.apply(lambda x: int((x[0].split(' ')[1]).split(':')[1]))
    df['second'] = tmp_time.apply(lambda x: int((x[0].split(' ')[1]).split(':')[2]))
    
    df['weekday'] = tmp_time.apply(lambda x: int(pd.to_datetime(x[0]).weekday()))
    
    print("time stamp done")
    
    # There is a tag in Page content <article data-channel=""tech"">
    df['channel'] = df['Page content'].apply(lambda x: re.findall(r'<article data-channel="(.*?)"', x))
    df['channel'] = df['channel'].apply(lambda x: x[0] if len(x) > 0 else 'unknown')
    # change channel into categorical data
    df['channel'] = pd.Categorical(df['channel']).codes
    print("channel done")
    
    # concat df and df_tag
    df = pd.concat([df, df_tag], axis=1)
    print("Concat done")
    
    return df

new_df = feature_extractor(df)

new_df.to_csv('new_df.csv', index=False)

# Remove the Page content column
new_df = new_df.drop(columns=['Page content'])


df_test = pd.read_csv('./test.csv')
new_df_test = feature_extractor(df_test)
new_df_test = new_df_test.drop(columns=['Page content']) 

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sysadmin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/sysadmin/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


df_tag done
time stamp done
channel done
Concat done
df_tag done
time stamp done
channel done
Concat done


Build the classifier

For these such amount of features after applying BOW, I consider use Random Forest to build the classifier would be a good choice.
I select LGBM and XGBoost as the classifier and a VotingClassifier to combine them, with weights [1, 0.5].
I have tried further combine them with simple LogisticRegression, but the result hasn't been improved, So it just ends up with two-model-voting system.

For tuning the model I lower the learning rate to 0.008 to make the model more robust.
And, increase the number of estimators to 500, which is better enough to have the optimal result without overfitting,.
Apply regularization to achieve better generalizability.

For daily testing, set the training set and validation set as the first 80% and the last 20% of the data, respectively.
And for the "LAST SHOT", I use almost the whole data to train the model and predict the test data.

In [3]:
trans = ColumnTransformer(
    [('Author', CountVectorizer(tokenizer=tokenizer_stem_nostop, lowercase=False,dtype=float,token_pattern=None),'author')
     ,('Topic', CountVectorizer(tokenizer=tokenizer_stem_nostop, lowercase=False,dtype=float,token_pattern=None),'topic')
     ,('Title', CountVectorizer(tokenizer=tokenizer_stem_nostop, lowercase=False,dtype=float,token_pattern=None),'title'),
     ],
    n_jobs=-1,
    remainder='passthrough'
)

print(new_df.head(5))
print("new_df done !")

# This time using whole dataset, with preprocessing
# Id,Popularity,Page content
# split df into training and validation with ratio of 80:20

# shuffle the data
new_df = new_df.sample(frac=1).reset_index(drop=True)

df_train = new_df.iloc[:int(len(new_df)*0.999)]
df_valid = new_df.iloc[int(len(new_df)*0.999):]

x_train = df_train.drop(columns=['Popularity','Id'])
y_train = df_train['Popularity']
y_train = y_train.apply(lambda x: 1 if x == 1 else 0)

# df_train.to_csv('df_train.csv', index=False)

print("Start to train the Model ")

# lgbm
lgbm = Pipeline([('ct', trans),
                 ('clf', LGBMClassifier(n_estimators=500, verbosity=-1, learning_rate=0.008, reg_alpha=0.1, reg_lambda=0.1))])
lgbm.fit(x_train, y_train)

score = cross_val_score(lgbm, x_train, y_train, cv=5, scoring='roc_auc')
print("lgbm AUROC score")
print('%.3f (+/-%.3f)' % (score.mean(), score.std()))


# other models

# xgboost
xgboost = Pipeline([('ct', trans),
                    ('clf', XGBClassifier(verbosity=0, n_estimators=500, learning_rate = 0.01, reg_alpha=0.1, reg_lambda=0.1))])
xgboost.fit(x_train, y_train)

score = cross_val_score(xgboost, x_train, y_train, cv=5, scoring='roc_auc')
print("xgboost AUROC score")
print('%.3f (+/-%.3f)' % (score.mean(), score.std()))

# # SVM
# linear_svc = OneVsRestClassifier(LinearSVC())
# calibrated_clf = CalibratedClassifierCV(
#            estimator=linear_svc,
#            cv="prefit")
# svm = Pipeline([('ct',trans),
#                 ('clf', linear_svc)])
# svm.fit(x_train, y_train)
# # OneVsRestClassifier(LinearSVC())
# score = cross_val_score(svm, x_train, y_train, cv=5, scoring='roc_auc')
# print("svm AUROC score")
# print('%.3f (+/-%.3f)' % (score.mean(), score.std()))


# Logistic Regression
logit = Pipeline([('ct', trans),
                ('clf', LogisticRegression(solver='liblinear',max_iter=1000, C=0.1))])
# logit.fit(x_train, y_train)

# score = cross_val_score(logit, x_train, y_train, cv=5, scoring='roc_auc')
# print("logit AUROC score")
# print('%.3f (+/-%.3f)' % (score.mean(), score.std()))

# voting
voting = VotingClassifier([('lgbm', lgbm),('xgboost', xgboost),('logit',logit)],
                          voting='soft', weights=[1, 0.5,0])
voting.fit(x_train, y_train)

score = cross_val_score(voting, x_train, y_train, cv=5, scoring='roc_auc')
print("AUROC score")
print('%.3f (+/-%.3f)' % (score.mean(), score.std()))

# exit(0)


print(":Training done ! Validation start !")

# Predict the Popularity of test data
print("Predict the Popularity of test data")

pred_proba = lgbm.predict_proba(new_df_test.drop(columns=['Id']))[:,1]

# Save the prediction to a csv file
print("Write the prediction to a csv file")
df_test['Popularity'] = pred_proba
df_test.to_csv('submission.csv', columns=['Id', 'Popularity'], index=False)

      Id  Popularity  year  month  day  hour  minute  second  weekday  \
0  12822          -1  2013      5    3    14      25      22        4   
1  11291           1  2013      2   20    15       9      48        2   
2  21366           1  2013      4   16    22      11      57        1   
3  23680           1  2014      5   22    19      47       1        3   
4    121          -1  2014      2   25    23      19       8        1   

   channel          author  content_len  num_image  num_url  \
0       10   chelsea_stark         2206          1       13   
1       28  stan_schroeder         1696          2        8   
2       29   annie_colbert         1836          1       41   
3       31   brian_koerber         1572          0       31   
4       28    chris_taylor         1942          1       14   

                                               title  \
0  questing meets candy in this odd ascii browser...   
1  yahoo redesigns homepage, adds infinite scrolling   
2  10 powerful

/home/sysadmin/miniconda3/envs/myenv/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/sysadmin/miniconda3/envs/myenv/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/sysadmin/miniconda3/envs/myenv/lib/python3

xgboost AUROC score
0.594 (+/-0.008)


/home/sysadmin/miniconda3/envs/myenv/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/sysadmin/miniconda3/envs/myenv/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/sysadmin/miniconda3/envs/myenv/lib/python3

AUROC score
0.599 (+/-0.008)
:Training done ! Validation start !
Predict the Popularity of test data
Write the prediction to a csv file


Conclusion

In this competition, I have tried to extract the features from the data and build the classifier with Random Forest, LGBM, and XGBoost.
The result is not good enough, and just like what professor said, the most time-consuming part is to preprocess data and feature engineering lol.
The LGBM model do provide "Feature importance" to demonstrate the actual value of a designed feature, and I think it is a good way to further improve the model by eliminating some of the redundant feature to achieve better prediction.